# Проект по SQL

<b>Цель:</b> Анализ базы данных сервиса по чтнению книг

<b> Задачи: </b>

1) Посчитать, сколько книг вышло после 1 января 2000 года;

2) Для каждой книги посчитать количество обзоров и среднюю оценку;

3) Определить издательство, которое выпустило наибольшее число книг толще 50 страниц

4) Определить автора с самой высокой средней оценкой книг

5) Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config =  # название базы данных #убрал конфиденциальные данные для подключения к базе данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], db_config['pwd'], db_config['host'],\
        db_config['port'], db_config['db']) 
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Исследование первых строк таблиц

Таблица books

In [3]:
query = 'SELECT * FROM books limit 5'

In [4]:
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица authors

In [5]:
query = 'SELECT * FROM authors limit 5'
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица publishers

In [6]:
query = 'SELECT * FROM publishers limit 5'
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица ratings

In [7]:
query = 'SELECT * FROM ratings limit 5'
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица reviews

In [8]:
query = 'SELECT * FROM reviews limit 5'
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Выполнение задач

### Посчитаем, сколько книг вышло после 1 января 2000 года;

In [9]:
query = """
    SELECT Count(book_id) FROM books 
    WHERE publication_date>'2000-01-01'"""
pd.io.sql.read_sql(query, con = engine)

,count
0,819


819 книг вышло после 1 января 2000 года

###  Для каждой книги посчитаем количество обзоров и среднюю оценку

In [10]:
query = """
    SELECT b.title title_of_book, Count(distinct(review_id)) count_reviews, ROUND(AVG(rating),2) average_mark
    FROM  reviews as r
    RIGHT JOIN books b ON r.book_id =b.book_id
    RIGHT JOIN ratings rat ON rat.book_id =b.book_id
    GROUP BY b.book_id
    ORDER BY average_mark DESC, count_reviews DESC
"""
pd.io.sql.read_sql(query, con = engine)

,title_of_book,count_reviews,average_mark
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,The Big Bad Wolf (Alex Cross #9),2,5.00
4,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


###  Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

In [11]:
query = """
    SELECT publisher, COUNT(book_id) count_of_books
    FROM  publishers p
    JOIN books b ON b.publisher_id=p.publisher_id
    WHERE num_pages > 50
    GROUP BY publisher
    ORDER BY COUNT(book_id) DESC
    LIMIT 1
"""
pd.io.sql.read_sql(query, con = engine)

,publisher,count_of_books
0,Penguin Books,42


Издательство, выпустившее наибольшее число книг толще 50 страниц, - Penguin Books	

### Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками;

In [12]:
query = """
    SELECT tabl.author, AVG(tabl.average_mark) average_mark
    FROM (SELECT author, title, COUNT(rating) count_marks, AVG(rating) average_mark
        FROM  authors a
        JOIN books b ON a.author_id=b.author_id
        JOIN ratings r ON r.book_id=b.book_id
        GROUP BY author, title
        HAVING COUNT(rating)>=50) as tabl
    GROUP BY author
    ORDER BY average_mark DESC
    LIMIT 1
"""
pd.io.sql.read_sql(query, con = engine)

,author,average_mark
0,J.K. Rowling/Mary GrandPré,4.283844


Авторы с наиболее высокой средней оценкой книг - J.K. Rowling/Mary GrandPré 

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [13]:
query = """
    SELECT AVG(count_reviews)
    FROM
    (SELECT r.username, COUNT(review_id) count_reviews
    FROM reviews r
    JOIN 
    (SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id)>50) as t ON r.username = t.username
    GROUP BY r.username) as g
    
"""
pd.io.sql.read_sql(query, con = engine)

,avg
0,24.333333


Около 24 обзоров оставили пользователи, которые поставили больше 50 оценок

<b> Выводы: </b>

1) 819 книг вышло после 1 января 2000 года

2) для каждой книги посчитали количество обзоров и среднюю оценку

3) Penguin Books - издательство, выпустившее наибольшее число книг толще 50 страниц

4) J.K. Rowling/Mary GrandPré - авторы с наиболее высокой средней оценкой книг

5) Около 24 обзоров оставили пользователи, которые поставили больше 50 оценок